In [1]:
# Main Libraries
import pandas as pd 
import numpy as np
from numpy import *

#Nice graphing tools
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns
import plotly
import plotly.offline as py
import plotly.tools as tls
import plotly.graph_objs as go
import plotly.tools as tls

from multiprocessing import Pool
from itertools import product
import knntsp
import warnings
import importlib
warnings.filterwarnings('ignore')

NUM_CORES = 12
PREPROCESSED_PATH = './../preprocessed/'

In [2]:
# Reading groupped data
def read_groupped_data(filename):
    timeSeries = pd.read_csv(filename,index_col=['Unnamed: 0'], parse_dates=['Unnamed: 0'])
    timeSeries.rename(columns=lambda x: eval(x), inplace=True)
    timeSeries.index = pd.to_datetime(timeSeries.index)
    return timeSeries

In [3]:
# Reading time series data
timeSeries = read_groupped_data(PREPROCESSED_PATH + 'timeseries.csv')

In [4]:
# Removing time series without data
timeSeries = timeSeries.dropna(axis=1)

In [5]:
W_TO_TEST = [3,4,5] #, 27
K_TO_TEST = [1,2,3,4,5]
RANDOM_STATE = 1
TRAIN_SIZE = 0.86
FOR_TEST_PREDICTION = 20

In [12]:
importlib.reload(knntsp)

FRAC = 0.1

results = []
X = timeSeries.T.sample(frac=FRAC, replace=False, random_state=RANDOM_STATE).T.copy().iloc[:-FOR_TEST_PREDICTION]

to_run = []
for w in W_TO_TEST:
    for k in K_TO_TEST:

        for column in X.columns:
            to_run.append([X, column, w, k, TRAIN_SIZE])
            
with Pool(NUM_CORES) as pool:
    results = pool.starmap(knntsp.predict_with_knn, to_run)

In [13]:
SAVE_SIMPLE_W_K_SELECTION = True

simple_w_k_result = pd.DataFrame()
for result in results:
    current_X, current_Y, mean_error, w, k, cluster_label = result
    simple_w_k_result = simple_w_k_result.append(pd.Series([w, k, mean_error[0]]), ignore_index=True)

simple_w_k_result.columns = ['w','k', 'mean_error']
groupped_mean = simple_w_k_result.groupby(['w','k'])['mean_error'].mean()

if SAVE_SIMPLE_W_K_SELECTION:
    pd.DataFrame([[groupped_mean.idxmin()[0], groupped_mean.idxmin()[1], groupped_mean.loc[groupped_mean.idxmin()]]], columns=['w', 'k', 'mean_error']).to_csv(PREPROCESSED_PATH + 'simple_knntsp_w_k.csv')

pd.DataFrame([[groupped_mean.idxmin()[0], groupped_mean.idxmin()[1], groupped_mean.loc[groupped_mean.idxmin()]]], columns=['w', 'k', 'mean_error'])

,w,k,mean_error
0,4.0,5.0,0.536526


In [6]:
importlib.reload(knntsp)
                 
cluster_labels = pd.read_csv(PREPROCESSED_PATH + 'standardization_based_cluster_labels.csv', header=-1, index_col=[0])
cluster_labels.columns = ['label']
cluster_labels.head()

CLUSTER_FRAC = 0.1

to_run = []
clustered_results = []
for name, group in cluster_labels.groupby(['label']):    
    group_sample = group.sample(frac=CLUSTER_FRAC, replace=False, random_state=RANDOM_STATE).copy()
    cluster_X = timeSeries.T.iloc[group_sample.index].T.iloc[:FOR_TEST_PREDICTION]    
    for w in W_TO_TEST:
        for k in K_TO_TEST:
            for column in cluster_X.columns:#[(16,21)]
                to_run.append([cluster_X, column, w, k, TRAIN_SIZE, False, name])

with Pool(NUM_CORES) as pool:
    clustered_results = pool.starmap(knntsp.predict_with_knn, to_run)

In [7]:
SAVE_CLUSTER_W_K_SELECTION = True

cluster_w_k_result = pd.DataFrame()
for result in clustered_results:
    current_X, current_Y, mean_error, w, k, cluster_label = result
    cluster_w_k_result = cluster_w_k_result.append(pd.Series([cluster_label, w, k, mean_error[0]]), ignore_index=True)

cluster_w_k_result.columns = ['cluster_label', 'w','k', 'mean_error']
cluster_groupped_mean = cluster_w_k_result.groupby(['cluster_label', 'w','k'], as_index=False)['mean_error'].mean()
cluster_groupped_mean = cluster_groupped_mean.iloc[cluster_groupped_mean.groupby(['cluster_label'])['mean_error'].idxmin().values]
cluster_groupped_mean.sort_values('cluster_label', inplace=True)
cluster_groupped_mean.reset_index(drop=True, inplace=True)

if SAVE_CLUSTER_W_K_SELECTION:
    cluster_groupped_mean.to_csv(PREPROCESSED_PATH + 'cluster_knntsp_w_k.csv')

cluster_groupped_mean.shape

(90, 4)

In [8]:
cluster_groupped_mean

,cluster_label,w,k,mean_error
0,1.0,3.0,2.0,0.058582
1,2.0,4.0,5.0,0.510510
2,3.0,4.0,5.0,0.443438
3,4.0,4.0,1.0,0.920857
4,5.0,3.0,3.0,0.626693
5,6.0,3.0,4.0,0.290514
6,7.0,4.0,5.0,0.802608
7,8.0,3.0,5.0,0.383788
8,9.0,5.0,5.0,1.189527
9,10.0,3.0,5.0,0.602407
